# SAT Calculations #

In [6]:
%load_ext autoreload
%autoreload 2
    
import os
import sys
import xarray as xr
import pandas as pd
import netCDF4 as nc
import numpy as np
import cftime
from tqdm import tqdm
from pathlib import Path
from dask.distributed import Client
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

sys.path.append('/home/563/sc1326/repos/cdrmip_extremes')
from cdrmip_extremes.configs import data_dir, models, expts
from cdrmip_extremes import load_data, sat

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
client = Client(memory_limit=None,threads_per_worker=1,n_workers=28)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/38563/status,
Dashboard: /proxy/38563/status,Workers: 28
Total threads: 28,Total memory: 0 B
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34917,Workers: 0
Dashboard: /proxy/38563/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:41817,Total threads: 1
Dashboard: /proxy/38173/status,Memory: 0 B
Nanny: tcp://127.0.0.1:37995,


## Load data ##

In [3]:
# load tas_anom
tas_anom = load_data.load_tas_anom()
tas_anom

{'ACCESS-ESM1-5': <xarray.Dataset> Size: 607MB
 Dimensions:  (lon: 180, lat: 90, time: 9360)
 Coordinates:
   * lon      (lon) float64 1kB 0.0 2.0 4.0 6.0 8.0 ... 352.0 354.0 356.0 358.0
   * lat      (lat) float64 720B -89.0 -87.0 -85.0 -83.0 ... 83.0 85.0 87.0 89.0
   * time     (time) object 75kB 0000-01-31 00:00:00 ... 0779-12-31 00:00:00
 Data variables:
     tas      (time, lat, lon) float32 607MB ...,
 'CanESM5': <xarray.Dataset> Size: 342MB
 Dimensions:  (lon: 180, lat: 90, time: 5280)
 Coordinates:
   * lon      (lon) float64 1kB 0.0 2.0 4.0 6.0 8.0 ... 352.0 354.0 356.0 358.0
   * lat      (lat) float64 720B -89.0 -87.0 -85.0 -83.0 ... 83.0 85.0 87.0 89.0
   * time     (time) object 42kB 0000-01-31 00:00:00 ... 0439-12-31 00:00:00
 Data variables:
     tas      (time, lat, lon) float32 342MB ...,
 'CESM2': <xarray.Dataset> Size: 264MB
 Dimensions:  (lon: 180, lat: 90, time: 4080)
 Coordinates:
   * lon      (lon) float64 1kB 0.0 2.0 4.0 6.0 8.0 ... 352.0 354.0 356.0 358.0
   

In [4]:
gwl_years = load_data.load_gwl_years()
gwl_years

{'ACCESS-ESM1-5': <xarray.DataArray (gwl: 3, branch: 2)> Size: 48B
 [6 values with dtype=int64]
 Coordinates:
   * gwl      (gwl) float64 24B 1.5 2.0 3.0
   * branch   (branch) <U9 72B 'ramp_up' 'ramp_down',
 'CanESM5': <xarray.DataArray (gwl: 3, branch: 2)> Size: 48B
 [6 values with dtype=int64]
 Coordinates:
   * gwl      (gwl) float64 24B 1.5 2.0 3.0
   * branch   (branch) <U9 72B 'ramp_up' 'ramp_down',
 'CESM2': <xarray.DataArray (gwl: 3, branch: 2)> Size: 48B
 [6 values with dtype=int64]
 Coordinates:
   * gwl      (gwl) float64 24B 1.5 2.0 3.0
   * branch   (branch) <U9 72B 'ramp_up' 'ramp_down',
 'CNRM-ESM2-1': <xarray.DataArray (gwl: 3, branch: 2)> Size: 48B
 [6 values with dtype=int64]
 Coordinates:
   * gwl      (gwl) float64 24B 1.5 2.0 3.0
   * branch   (branch) <U9 72B 'ramp_up' 'ramp_down',
 'GFDL-ESM4': <xarray.DataArray (gwl: 3, branch: 2)> Size: 48B
 [6 values with dtype=int64]
 Coordinates:
   * gwl      (gwl) float64 24B 1.5 2.0 3.0
   * branch   (branch) <U9 72B 'ra

### Extract mean temperatures over GWL periods (21-year average centred on time of GWL crossing) ###

In [7]:
gwl_periods = {}
for model, ds in tas_anom.items():
    gwl_periods[model] = sat.extract_gwl_period(
        ds,
        gwl_years[model],
        21,
        time_dim='time'
    )
gwl_periods
        

{'ACCESS-ESM1-5': <xarray.Dataset> Size: 43MB
 Dimensions:  (branch: 2, gwl: 3, year: 111, lat: 90, lon: 180)
 Coordinates:
   * lon      (lon) float64 1kB 0.0 2.0 4.0 6.0 8.0 ... 352.0 354.0 356.0 358.0
   * lat      (lat) float64 720B -89.0 -87.0 -85.0 -83.0 ... 83.0 85.0 87.0 89.0
   * year     (year) int64 888B 51 52 53 54 55 56 57 ... 266 267 268 269 270 271
   * gwl      (gwl) float64 24B 1.5 2.0 3.0
   * branch   (branch) <U9 72B 'ramp_up' 'ramp_down'
 Data variables:
     tas      (branch, gwl, year, lat, lon) float32 43MB 1.391 1.392 ... nan nan,
 'CanESM5': <xarray.Dataset> Size: 42MB
 Dimensions:  (branch: 2, gwl: 3, year: 109, lat: 90, lon: 180)
 Coordinates:
   * lon      (lon) float64 1kB 0.0 2.0 4.0 6.0 8.0 ... 352.0 354.0 356.0 358.0
   * lat      (lat) float64 720B -89.0 -87.0 -85.0 -83.0 ... 83.0 85.0 87.0 89.0
   * year     (year) int64 872B 39 40 41 42 43 44 45 ... 288 289 290 291 292 293
   * gwl      (gwl) float64 24B 1.5 2.0 3.0
   * branch   (branch) <U9 72B 'ra

### Extract mean temperatures over final 21-year period and equivalent ramp-up GWL ###

In [9]:
# load
match_ds = load_data.load_equiv_gwls()
match_ds

[autoreload of cdrmip_extremes.load_data failed: Traceback (most recent call last):
  File "/g/data/xp65/public/apps/med_conda/envs/analysis3-25.08/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/g/data/xp65/public/apps/med_conda/envs/analysis3-25.08/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "/g/data/xp65/public/apps/med_conda/envs/analysis3-25.08/lib/python3.11/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 621, in _exec
  File "<frozen importlib._bootstrap_external>", line 936, in exec_module
  File "<frozen importlib._bootstrap_external>", line 1074, in get_code
  File "<frozen importlib._bootstrap_external>", line 1004, in source_to_code
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_rem

AttributeError: module 'cdrmip_extremes.load_data' has no attribute 'load_equiv_gwls'

### Calculate differences between ramp-up and ramp-down periods ###